# Supplementary figure on new odor recognition
Show that the response to the mixture is more similar to the new odor than to background odors after habituation. And other panels to supplement figure 2. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
_, n_neu, n_orn = np.load(pj(data_folder, "sample_2d_simulation.npz"))["mbarser"].shape

In [ ]:
# Extra aesthetic parameters for this figure
# More legend rcParams: make everything smaller by 30 %
#plt.rcParams["patch.linewidth"] = 0.75
#legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
#             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
#for k in legend_rc:
#    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

neuron_colors = neuron_colors_full24[[8, 17]]
new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

# Similarity to background vs to new odor
In turbulent background statistics

Unsure where to put this panel, likely with the eigenvalues plot. 

This is not only IBCM: we have the distribution for all models. 

In [ ]:
jaccards_back = np.load(pj(data_folder, "jaccard_similarities_back_identity.npz"))
# Shape of the Jaccards array: sim_id, n_new_odors, n_times, n_new_concs, n_back_samples
# Jaccard similarity to new odor alone vs similarity to most similar background odor
jaccards_back_max = {a:np.amax(jaccards_back[a], axis=-1) for a in jaccards_back}
jaccards_new = np.load(pj(data_folder, "jaccard_similarities_identity.npz"))

In [ ]:
all_dists = np.load(pj(data_folder, "new_mix_distances_identity.npz"))
new_concs = all_dists["new_concs"]
rel_new_concs = [a/new_concs[1] for a in new_concs]
del all_dists

In [ ]:
# Scatter plot of Jaccards with new odor vs with background
# Plot model histogram results
# One plot per new odor concentration
n_new_concs = jaccards_back_max["ibcm"].shape[3]
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.7, plt.rcParams["figure.figsize"][1]*1.1)
axes = axes.flatten()
models2 = ["none", "optimal", "avgsub", "biopca", "ibcm", "orthogonal"]
for m in models2:  # Plot IBCM last
    all_jacs = np.median(jaccards_new[m], axis=(2,4))  # Median time point, all back samples there
    all_jacs_back = np.median(jaccards_back_max[m], axis=(2,))
    for i in range(n_new_concs):
        axes[i].scatter(all_jacs_back[::6, ::6, i].flatten(), all_jacs[::6, ::6, i].flatten(), 
            s=9.0, color=model_colors.get(m), alpha=0.5, label=model_nice_names[m])
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    conc_rel = rel_new_concs[i]
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", color="grey", lw=1.0, zorder=0)
    axes[i].set_title(r"New conc.$= {:.1f} \langle c \rangle$".format(conc_rel))
    axes[i].set_xlabel("Jaccard with most\nsimilar background odor")
    axes[i].set_ylabel("Jaccard with new odor")

# Place legend after tight_layout, so it is outside of the area occupied by the other plot too
fig.tight_layout()
leg = axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)


if do_save_plots:
    fig.savefig(pj(panels_folder, "turbulent_recognition_jaccards_with_back_new_scatter.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,)
    )

plt.show()
plt.close()

In [ ]:
def hist_outline(ax, bins, height, **plot_kwargs):
    plot_hist = np.stack([height, height], axis=1).flatten()
    plot_edges = np.stack([bins[:-1], bins[1:]], axis=1).flatten()
    ax.plot(plot_edges, plot_hist, **plot_kwargs)
    ax.fill_between(plot_edges, min(0.0, height.min()), plot_hist,
                    color=plot_kwargs.get("color"), alpha=0.3)
    return ax

In [ ]:
# Compute interesting statistics
jac_histograms = {}
jac_cdfs = {}
jac_stats = {}
for m in models2:
    jac_histograms[m] = {}
    jac_cdfs[m] = {}
    jac_stats[m] = {}
    for i in range(len(new_concs)):
        conc = new_concs[i]
        jacs_sim = jaccards_back_max[m][:, :, :, i].flatten()
        jac_histograms[m][conc] = np.histogram(jacs_sim, bins="doane", density=True)
        jacs_dists = 1.0 - jacs_sim
        # There is only a discrete number of possible J, increments of card(z_n \cap z_mix)
        # So count each value
        dists_axis, dists_counts = np.unique(jacs_dists, return_counts=True)
        reorder = np.argsort(dists_axis)
        dists_axis = dists_axis[reorder]
        dists_counts = dists_counts[reorder] / jacs_dists.size
        dists_cdf = np.cumsum(dists_counts) 
        jac_cdfs[m][conc] = dists_cdf, dists_axis
        jac_stats[m][conc] = [
            np.mean(jacs_sim), 
            np.median(jacs_sim), 
            np.var(jacs_sim),
        ]

In [ ]:
# Scatter plot of Jaccards with new odor vs with background
# Plot model histogram results
# One plot per new odor concentration
n_new_concs = jaccards_back_max["ibcm"].shape[3]
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.75, plt.rcParams["figure.figsize"][1]*1.1)
axes = axes.flatten()
models2 = ["none", "optimal", "avgsub", "biopca", "ibcm", "orthogonal"]
for m in models2:  # Plot IBCM last
    all_jacs_back = jaccards_back_max[m]
    print(all_jacs.shape)
    for i in range(n_new_concs):
        lbl = model_nice_names.get(m) if i == 0 else ""
        conc = new_concs[i]
        hist_outline(axes[i], jac_histograms[m][conc][1], jac_histograms[m][conc][0],
            color=model_colors[m], lw=1.0, label=lbl)
        #lbl = "Median" if i == 1 and m == "none" else ""
        #axes[i].axvline(jac_stats[m][conc][1], ls="--", color=model_colors[m], 
        #           lw=0.75, label=lbl)
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    conc_rel = rel_new_concs[i]
    axes[i].set_title(r"New conc.$= {:.1f} \langle c \rangle$".format(conc_rel))
    axes[i].set_xlabel("Jaccard with background\n(smaller is better)")
    axes[i].set_ylabel("Probability density")
axes[0].legend(loc="upper right", bbox_to_anchor=(1.0, 1.0), frameon=False)
#axes[1].legend(loc="upper right", frameon=False)
fig.tight_layout()

if do_save_plots:  # Marginal similarity to background
    fig.savefig(pj(panels_folder, "turbulent_recognition_jaccards_with_back_histograms.pdf"), 
            transparent=True, bbox_inches="tight"
    )

plt.show()
plt.close()

# Supplementary panels on correlation with distance to the background

In [ ]:
show_models = ["none", "avgsub", "biopca", "ibcm", "optimal"]

In [ ]:
new_back_dists = np.load(pj(data_folder, "new_back_distances_identity.npz"))["new_back_distances"]

In [ ]:
# Concatenated Jaccard scores shaped [n_background, n_new_odors, n_times, n_new_concs, n_back_samples]
# new_back_dists shaped background, new_odor
# Just need median along axes 2, 3, 4 to get one per [back, new] pair
jaccards_per_pair = {}
for m in show_models:
    jaccards_per_pair[m] = np.median(jaccards_new[m], axis=(2, 3, 4))

In [ ]:
# Warning: kdeplot is slow, this figure takes around 1 min to generate
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.7, plt.rcParams["figure.figsize"][1]*1.1)
ax = axes.flat[0]
plot_type = "scatter"  # or "kde"
if plot_type == "kde":
    data = pd.concat({m:pd.DataFrame(np.stack([jaccards_per_pair[m].flatten(), new_back_dists.flatten()], axis=1), 
                                columns=pd.Index(["jaccard", "new-back"]))
                    for m in show_models}, names=["Model"])
    data = data.rename(model_nice_names, level="Model")
    model_nice_colors = {model_nice_names[m]:model_colors[m] for m in show_models}
    g = sns.kdeplot(data=data.reset_index(), x="new-back", y="jaccard", hue="Model", 
            palette=model_nice_colors, ax=ax, fill=True, alpha=0.5)
    sns.move_legend(g, frameon=False, loc="upper left")
elif plot_type == "scatter":
    for m in show_models:
        xy = np.stack([new_back_dists.flatten(), jaccards_per_pair[m].flatten()], axis=0)
        xy = np.unique(xy, axis=1)  # Find unique (x, y) pairs
        ax.scatter(xy[0], xy[1], color=model_colors[m], label=model_nice_names[m], s=0.9, alpha=0.1)

ax.set(xlabel=r"New odor orthogonal part, $\|\mathbf{s}_\mathrm{new, \perp}\|$", 
       ylabel=r"Jaccard similarity $(z_{\mathrm{new}}, z_{\mathrm{mix}})$")
handles, labels = ax.get_legend_handles_labels()
handles_new = []
for h in handles:
    #handles_new.append(mpl.patches.Patch(facecolor=h.get_facecolor(), edgecolor=h.get_facecolor()))
    handles_new.append(mpl.lines.Line2D([0], [0], marker='o', color=h.get_facecolor(), 
                          markerfacecolor=h.get_facecolor(), markersize=4, alpha=1.0, ls="none"))
ax.legend(handles_new[::-1], labels[::-1], fontsize=6, frameon=False, handlelength=1.0, handletextpad=0.3, 
          borderaxespad=0.3, labelspacing=0.3)

# Also, in particular, the improvement afforded by IBCM compared to no habituation
improvements = jaccards_per_pair["ibcm"] - jaccards_per_pair["none"]
fraction_positive = (improvements >= 0).sum() / improvements.size

ax = axes.flat[1]
ax.scatter(new_back_dists.flatten(), improvements.flatten(), s=1.0, alpha=0.7, color=model_colors["ibcm"])

ax.axhline(0.0, ls="--", color="grey", zorder=2)
ax.set(xlabel=r"New odor orthogonal part, $\|\mathbf{s}_\mathrm{new, \perp}\|$", 
       ylabel="IBCM vs None improvement")
ax.annotate(f"{fraction_positive*100:.1f} %", xy=(0.8, 0.02), va="bottom")
ax.annotate(f"{(1.0-fraction_positive)*100:.1f} %", xy=(0.8, -0.04), va="top")


fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_recognition_back-new_distance_jaccard_correlation.png"), 
                dpi=300, transparent=True, bbox_inches="tight")
plt.show()
plt.close()